# Grapher

Grapher is an optional compiler component which serializes the AST (Abstract Syntax Tree) using [graph description language](https://en.wikipedia.org/wiki/DOT_(graph_description_language)) and generates an image of it.

![pp-01](https://i.postimg.cc/SNmFQ6X0/pp-01.png)

Klasa **Node** predstavlja baznu klasu za formiranje AST, a klase koje je nasleđuju odgovaraju svakoj ispravnoj semantičkoj strukturi.

In [ ]:
class Node():
    pass

class Program(Node):
    def __init__(self, block):      
        self.block = block

class Block(Node):
    def __init__(self, decl_block, compound_statement):
        self.decl_block = decl_block
        self.compound_statement = compound_statement

class DeclBlock(Node):
    def __init__(self, nodes):      
        self.nodes = nodes

class VarBlock(Node):
    def __init__(self, nodes):      
        self.nodes = nodes

class VarDecl(Node):                   
    def __init__(self, type_, id_, assign):
        self.type_ = type_
        self.id_ = id_
        self.assign = assign

class StringDecl(Node):                   
    def __init__(self, type_, id_, size):
        self.type_ = type_
        self.id_ = id_
        self.size = size

class ArrayDecl(Node):
    def __init__(self, type_, id_, low, high, elems):
        self.type_ = type_
        self.id_ = id_
        self.low = low
        self.high = high
        self.elems = elems

class ProcDecl(Node):
    def __init__(self, id_, params, block):
        self.id_ = id_
        self.params = params
        self.block = block

class FuncDecl(Node):
    def __init__(self, type_, id_, params, block):
        self.type_ = type_
        self.id_ = id_
        self.params = params
        self.block = block

class CompoundStatement(Node):
    def __init__(self, nodes):
      self.nodes = nodes

class If(Node):
    def __init__(self, cond, true, false):
        self.cond = cond
        self.true = true
        self.false = false

class For(Node):                        # down to obraditi
    def __init__(self, init, limit, step, compound_statement):
        self.init = init
        self.limit = limit
        self.step = step
        self.compound_statement = compound_statement

class While(Node):
    def __init__(self, cond, compound_statement): 
        self.cond = cond
        self.compound_statement = compound_statement

class RepeatUntil(Node):
    def __init__(self, cond, statements):
        self.cond = cond
        self.statements = statements

class FuncCall(Node):
    def __init__(self, id_, args):
        self.id_ = id_
        self.args = args

class ProcCall(Node):
    def __init__(self, id_, args):
        self.id_ = id_
        self.args = args

class Assign(Node):
    def __init__(self, id_, expr): 
        self.id_ = id_
        self.expr = expr

class Params(Node):
    def __init__(self, params):
        self.params = params

class VarParam(Node):                   
    def __init__(self, type_, id_):
        self.type_ = type_
        self.id_ = id_

class ValueParam(Node):                   
    def __init__(self, type_, id_):
        self.type_ = type_
        self.id_ = id_

class Args(Node):
    def __init__(self, args):
        self.args = args

class ArrayElem(Node):
    def __init__(self, id_, index):
        self.id_ = id_
        self.index = index

class Elems(Node):      
    def __init__(self, elems):
        self.elems = elems

class Break(Node):
    pass

class Continue(Node):
    pass

class Exit(Node):
    pass

class Type(Node):                       # int, char etc
    def __init__(self, value):
        self.value = value

class Int(Node):                        # int value
    def __init__(self, value):
        self.value = value

class Real(Node):
    def __init__(self, value):
        self.value = value

class Boolean(Node):
    def __init__(self, value):
        self.value = value

class Char(Node):
    def __init__(self, value):
        self.value = value

class String(Node):
    def __init__(self, value):
        self.value = value

class Id(Node):
    def __init__(self, value):
        self.value = value

class BinOp(Node):
    def __init__(self, first, symbol, second):
        self.first = first
        self.symbol = symbol
        self.second = second

class UnOp(Node):
    def __init__(self, symbol, first):      
        self.symbol = symbol
        self.first = first

**Visitor** class represent a basis for the AST traversal.

Method **visit** searches the current object for a method that corelates with the type of the passed node.

Method **die** is used if the wanted method doesn't exist (when we try to access an unhandled node).


In [ ]:
class Visitor():
    def visit(self, parent, node):
        method = 'visit_' + type(node).__name__
        visitor = getattr(self, method, self.die)
        return visitor(parent, node)

    def die(self, parent, node):
        method = 'visit_' + type(node).__name__
        raise SystemExit("Missing method: {}".format(method))

 [Graphviz](https://graphviz.org) tool for AST serialization.

In [ ]:
!apt install -y graphviz
!pip install graphviz

Modules needed for AST serialization.

In [ ]:
from graphviz import Digraph, Source
from IPython.display import Image

**Grapher** class contains methods for AST traversal forming a [digraph](https://en.wikipedia.org/wiki/Directed_graph) which will be shown in PNG format.

Method **graph** forms a graphic representation of the AST by recursive calling of the **visit** method.

Method **add_node** adds a node and an edge to the AST by calling the **add_edge** method.

Method **add_edge** adds a directed egde to the AST from **parent** node to **node**.

In [ ]:
class Grapher(Visitor):
    def __init__(self, ast):
        self.ast = ast
        self._count = 1
        self.dot = Digraph()
        self.dot.node_attr['shape'] = 'box'
        self.dot.node_attr['height'] = '0.1'
        self.dot.edge_attr['arrowsize'] = '0.5'

    def add_node(self, parent, node, name=None):
        node._index = self._count
        self._count += 1
        caption = type(node).__name__
        if name is not None:
            caption = '{} : {}'.format(caption, name)
        self.dot.node('node{}'.format(node._index), caption)
        if parent is not None:
            self.add_edge(parent, node)

    def add_edge(self, parent, node):
        src, dest = parent._index, node._index
        self.dot.edge('node{}'.format(src), 'node{}'.format(dest))

    def visit_Program(self, parent, node):
        self.add_node(parent, node)
        self.visit(node, node.block)
    
    def visit_Block(self, parent, node):
        self.add_node(parent, node)
        for d in node.decl_block:
            self.visit(node, d)
        #self.visit(node, node.decl_block)
        self.visit(node, node.compound_statement)
    '''
    def visit_DeclBlock(self, parent, node):
        self.add_node(parent, node)
        for n in node.nodes:
            self.visit(node, n)
    '''
    def visit_VarBlock(self, parent, node):
        self.add_node(parent, node)
        for n in node.nodes:
            self.visit(node, n)

    def visit_VarDecl(self, parent, node):
        self.add_node(parent, node)
        self.visit(node, node.type_)
        self.visit(node, node.id_)
        if node.assign is not None:
            self.visit(node, node.assign)

    def visit_StringDecl(self, parent, node):
        self.add_node(parent, node)
        self.visit(node, node.type_)
        self.visit(node, node.id_)
        self.visit(node, node.size)

    def visit_ArrayDecl(self, parent, node):
        self.add_node(parent, node)
        self.visit(node, node.type_)
        self.visit(node, node.id_)
        if node.low is not None:
            self.visit(node, node.low)
        if node.high is not None:
            self.visit(node, node.high)
        if node.elems is not None:
            self.visit(node, node.elems)

    def visit_ProcDecl(self, parent, node):
        self.add_node(parent, node)
        self.visit(node, node.id_)
        self.visit(node, node.params)
        self.visit(node, node.block)
    
    def visit_FuncDecl(self, parent, node):
        self.add_node(parent, node)
        self.visit(node, node.type_)
        self.visit(node, node.id_)
        self.visit(node, node.params)
        self.visit(node, node.block)

    def visit_CompoundStatement(self, parent, node):
        self.add_node(parent, node)
        for n in node.nodes:
            self.visit(node, n)

    def visit_If(self, parent, node):
        self.add_node(parent, node)
        self.visit(node, node.cond)
        self.visit(node, node.true)
        if node.false is not None:
            self.visit(node, node.false)

    def visit_For(self, parent, node):
        self.add_node(parent, node)
        self.visit(node, node.init)
        self.visit(node, node.limit)
        self.visit(node, node.step)
        self.visit(node, node.compound_statement)

    def visit_While(self, parent, node):
        self.add_node(parent, node)
        self.visit(node, node.cond)
        self.visit(node, node.block)

    def visit_RepeatUntil(self, parent, node):
        self.add_node(parent, node)
        self.visit(node, node.cond)
        self.visit(node, node.statements)

    def visit_FuncCall(self, parent, node):
        self.add_node(parent, node)
        self.visit(node, node.id_)
        self.visit(node, node.args)

    def visit_ProcCall(self, parent, node):
        self.add_node(parent, node)
        self.visit(node, node.id_)
        self.visit(node, node.args)

    def visit_Assign(self, parent, node):
        self.add_node(parent, node)
        self.visit(node, node.id_)
        self.visit(node, node.expr)

    def visit_Params(self, parent, node):
        self.add_node(parent, node)
        for p in node.params:
            self.visit(node, p)

    def visit_VarParam(self, parent, node):
        self.add_node(parent, node)
        self.visit(node, node.type_)
        self.visit(node, node.id_)

    def visit_ValueParam(self, parent, node):
        self.add_node(parent, node)
        self.visit(node, node.type_)
        self.visit(node, node.id_)

    def visit_Args(self, parent, node):
        self.add_node(parent, node)
        for a in node.args:
            self.visit(node, a)

    def visit_Elems(self, parent, node):
        self.add_node(parent, node)
        for e in node.elems:
            self.visit(node, e)
            
    def visit_ArrayElem(self, parent, node):
        self.add_node(parent, node)
        self.visit(node, node.id_)
        self.visit(node, node.index)

    def visit_Break(self, parent, node):
        self.add_node(parent, node)

    def visit_Continue(self, parent, node):
        self.add_node(parent, node)

    def visit_Exit(self, parent, node):
        self.add_node(parent, node)

    def visit_Type(self, parent, node):
        name = node.value
        self.add_node(parent, node, name)

    def visit_Int(self, parent, node):
        name = node.value
        self.add_node(parent, node, name)
    
    def visit_Real(self, parent, node):
        name = node.value
        self.add_node(parent, node, name)
    
    def visit_Boolean(self, parent, node):
        name = node.value
        self.add_node(parent, node, name)

    def visit_Char(self, parent, node):
        name = node.value
        self.add_node(parent, node, name)

    def visit_String(self, parent, node):
        name = node.value
        self.add_node(parent, node, name)

    def visit_Id(self, parent, node):
        name = node.value
        self.add_node(parent, node, name)

    def visit_BinOp(self, parent, node):
        name = node.symbol
        self.add_node(parent, node, name)
        self.visit(node, node.first)
        self.visit(node, node.second)

    def visit_UnOp(self, parent, node):
        name = node.symbol
        self.add_node(parent, node, name)
        self.visit(node, node.first)

    def graph(self):
        self.visit(None, self.ast)
        s = Source(self.dot.source, filename='graph', format='png')
        return s.view()